## <4-2 문서 분류 모델 학습하기>  

문서 분류 모델의 데이터 전처리와 학습 과정을 실습으로 진행해 보겠습니다.

### 영화 리뷰 감성 분석 모델 만들기
---

<**1단계**> **코랩 노트북 초기화**  
이번 실습은 웹 브라우저에서 다음 주소(bit.ly/3FjJJ1H)에 접속하면 코랩 환경에서 수행할 수 있습니다. 이전 실습과 마찬가지로 코랩에서 **[내 드라이브에 복사]**를 진행합니다.

모델을 파인튜닝하려면 하드웨어 가속기를 사용해야 계산 속도를 높일 수 있습니다. 코랩에서는 GPU와 TPU 두 종류의 가속기를 지원합니다. 코랩의 메뉴에서 [런타임 $\rightarrow$ **런타임 유형 변경**]을 클릭하고 **[GPU]나 [TPU] 둘 중 하나를 선택**합니다.

<center><그림 4-3 하드웨어 가속기 설정></center>

<p align="center"><img src="https://i.imgur.com/i4XvOhQ.png">  

<center>출처 : ratsgo's NLPBOOK</center>


**[None]**을 선택해 하드웨어 가속 기능을 사용하지 않으면 파인튜닝 속도가 급격히 느려지므로 반드시 GPU나 TPU 둘 중 하나를 사용하세요. 참고로 TPU 학습은 라이브러리 지원 등이 GPU보다 불안정한 편이므로 될 수 있으면 GPU를 사용하기 바랍니다.

<**2단계**> **각종 설정하기**  
1단계 코랩 노트북 초기화 과정에서 하드웨어 가속기로 TPU를 선택했다면 다음 코드를 실행하세요. 그러면 TPU 관련 라이브러리들을 설치합니다. 다만 GPU를 선택했다면 이 코드를 생략하고 코드 4-2부터 실행하세요.  

cf) 구글 코랩 환경은 예고 없이 수시로 업데이트될 수 있으므로 언젠가 코드가 동작하지 않을 수도 있습니다. TPU를 사용하기 위한 최신 패키지 버전을 확인하려면 다음 링크를 확인하세요(Installing PyTorch/XLA 절 참고).  

In [ ]:
#코드 4-1 TPU 관련 패키지 설치
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

다음 코드는 TPU 이외에 의존성 있는 패키지를 설치합니다.  
명령어 맨 앞에 붙은 느낌표는 코랩 환경에서 파이썬이 아닌, 셸(shell) 명령을 수행한다는 의미입니다.

In [1]:
#코드 4-2 의존성 패키지 설치
!pip install ratsnlp

     |████████████████████████████████| 42 kB 365 kB/s 
     |████████████████████████████████| 2.8 MB 8.4 MB/s 
     |████████████████████████████████| 57 kB 4.0 MB/s 
     |████████████████████████████████| 806 kB 7.4 MB/s 
     |████████████████████████████████| 134 kB 34.9 MB/s 
     |████████████████████████████████| 636 kB 35.3 MB/s 
     |████████████████████████████████| 829 kB 30.3 MB/s 
     |████████████████████████████████| 397 kB 45.2 MB/s 
     |████████████████████████████████| 67 kB 4.2 MB/s 
     |████████████████████████████████| 3.3 MB 33.6 MB/s 
     |████████████████████████████████| 895 kB 40.7 MB/s 
     |████████████████████████████████| 1.1 MB 41.4 MB/s 
     |████████████████████████████████| 96 kB 5.1 MB/s 
     |████████████████████████████████| 271 kB 45.8 MB/s 
     |████████████████████████████████| 94 kB 2.8 MB/s 
     |████████████████████████████████| 144 kB 50.0 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha2

코랩 노트북은 일정 시간 사용하지 않으면 당시까지의 모든 결과물이 날아갈 수 있습니다. 모델 체크포인트 등을 저장해 두기 위해 자신의 구글 드라이브를 코랩 노트북과 연결합니다.  
다음 코드를 실행하면 됩니다.

In [2]:
#코드 4-3 구글 드라이브와 연결
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


이번 실습에서는 **kcbert-base** 모델을 NSMC 데이터로 파인튜닝해 볼 예정입니다.  
다음 코드를 실행하면 관련 설정을 할 수 있습니다.

In [4]:
#코드 4-4 모델 환경 설정
import torch
from ratsnlp.nlpbook.classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="nsmc",
    downstream_model_dir="/gdrive/My drive/nlpbook/checkpoint-doccls",
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    max_seq_length=128,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

참고로 **TrainArguments**의 각 인자가 하는 역할과 의미는 다음과 같습니다.  
>
>- **pretrained_model_name**: 프리트레인 마친 언어 모델의 이름(단, 해당 모델은 허깅페이스 모델 허브에 등록되어 있어야 합니다)  
- **downstream_corpus_name**: 다운스트림 데이터의 이름  
- **downstream_corpus_root_dir**: 다운스트림 데이터를 내려받을 위치. 입력하지 않으면 /root/Korpora에 저장됩니다.  
- **downstream_model_dir**: 파인튜닝된 모델의 체크포인트가 저장될 위치. /gdrive/My Drive/nlpbook/checkpoint-doccls로 지정하면 자신의 구글 드라이브 [내 폴더] 아래 nlpbook/checkpoint-doccls 디렉터리에 저장됩니다.  
- **batch_size**: 배치 크기. 하드웨어 가속기로 GPU를 선택(torch.cuda.is_available()==True)했다면 32, TPU라면(torch.cuda.is_available()==False) 4. 코랩 환경에서 TPU는 보통 8개 코어가 할당되는데 batch_size는 코어별로 적용되는 배치 크기이므로 이렇게 설정해 둡니다.  
- **learning_rate**: 러닝 레이트(보폭). 1회 스텝에서 모델을 얼마나 업데이트할지에 관한 크기를 가리킵니다. 이와 관련한 자세한 내용은 <3-4 아담 옵티마이저> 부분을 참고하세요.  
- **max_seq_length**: 토큰 기준 입력 문장 최대 길이. 이보다 긴 문장은 max_seq_length로 자르고, 짧은 문장은 max_seq_length가 되도록 스페셜 토큰 ([PAD])을 붙여 줍니다.  
- **epochs**: 학습 에포크 수. 3이라면 학습 데이터 전체를 3회 반복 학습합니다.  
- **tpu_cores**: TPU 코어 수. 하드웨어 가속기로 GPU를 선택했다면 0, TPU라면 8  
- **seed**: 랜덤 시드(정수). None를 입력하면 랜덤 시드를 고정하지 않습니다.

다음 코드를 실행해 랜덤 시드를 설정합니다. 이 코드는 **args**에 지정된 시드로 고정하는 역할을 합니다.

In [10]:
#코드 4-5 랜덤 시드 고정
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


무작위 수를 의미하는 ***난수(random number)*** 는 배치를 뽑거나 드롭아웃 대상 뉴런의 위치를 정할 때 등 다양하게 쓰입니다. 컴퓨터는 난수 생성 알고리즘으로 무작위 수를 만들어 내는데요, 이때 난수 생성 알고리즘을 실행하기 위해 쓰는 수를 ***랜덤 시드(random seed)*** 라고 합니다. 만일 같은 시드를 사용한다면 컴퓨터는 계속 같은 패턴의 난수를 생성합니다.

In [11]:
#코드 4-6 로거 설정
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='document-classification', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My drive/nlpbook/checkpoint-doccls', max_seq_length=128, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=2, fp16=False, tpu_cores=0)


<**3단계**> **말뭉치 내려받기**  
다음 코드를 실행하면 NSMC 데이터를 내려받습니다. 데이터를 내려받는 도구로 코포라(korpora)라는 파이썬 오픈소스 패키지를 사용해, **corpus_name(nsmc)**에 해당하는 말뭉치를 **root_dir**(/root/Korpora)아래에 저장해 둡니다.

In [12]:
#코드 4-7 말뭉치 내려받기
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=True,
)

[nsmc] download ratings_train.txt: 14.6MB [00:00, 82.2MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 38.1MB/s]                            


<**4단계**> **토크나이저 준비하기**  
이 책에서 다루는 데이터의 기본 단위는 텍스트 형태의 문장입니다. 토큰화란 문장을 토큰 시퀀스로 분절하는 과정을 가리킵니다. 이 실습에서 사용하는 모델은 자연어 문장을 분절한 토큰 시퀀스를 입력으로 받는데요, 다음 코드를 실행해 **kcbert-base** 모델이 사용하는 토크나이저를 선언합니다.  
토크나이저는 토큰화를 수행하는 프로그램이라는 뜻입니다.

In [13]:
#코드 4-8 토크나이저 준비
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

<**5단계**> **데이터 전처리하기**  
딥러닝 모델을 학습하려면 학습 데이터를 배치 단위로 계속 모델에 공급해 주어야 합니다. 파이토치에서는 이 역할을 ***데이터 로더(dataloader)*** 가 수행하는데요, 그 개념을 나타내면 다음 그림과 같습니다. 데이터 로더는 ***데이터셋(dataset)*** 이 보유하고 있는 인스턴스를 배치 크기만큼 뽑아서(sample) 자료형, 데이터 길이 등 정해진 형식에 맞춰(collate) 배치를 만들어 줍니다.

<center><그림 4-4 데이터 로더의 기본 구조></center>

<p align="center"><img src="https://i.imgur.com/bD07LbT.jpg">  

<center>출처 : ratsgo's NLPBOOK</center>

